In [1]:
%load_ext autoreload

In [ ]:
"""

Subjects articulate 36 distinct words that span the entire English language phonetic space in `silent' and `audible' manner. 


They are - 
0. Eager 
1. lift
2. eight
3. edge
4. cap
5. matted
6. tub
7. box
8. rune
9. rook
10. folder
11. block
12. fun
13. mop
14. pod
15. very
16. went
17. throat
18. this
19. tango
20. doubt
21. not
22. pretty
23. xerox
24. rodent
25. limb
26. batch
27. jeep
28. ship
29. beige
30. yes
31. echo
32. gold
33. sing
34. Uh-oh
35. hiccup

DATA is given in a numpy array of dimensions (360, 22, 7500) - (36 words each repeated 10 times, 22 channels, 7500 time samples).
Raw data was filtered using 3rd order Butterworth bandpass filter between 80 and 1000 Hertz. 
"""

In [2]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from spdLearning import spdNN
from spdLearning import optimizers 
from spdLearning import trainTest
from spdLearning import spdNet

In [3]:
class BaseDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return self.data[index].astype('float32'), self.labels[index]

    def __len__(self):
        return len(self.data)

In [4]:
dev = "cpu" 
device = torch.device(dev)

In [5]:
numberWords = 36
trialsPerWord = 10
numberTrials = numberWords * trialsPerWord
numberChannels = 22
windowLength = 7500

In [6]:
subjectNumber = 10
articulationManner = "Unvoiced"
subject = "Subject" + str(subjectNumber)

In [7]:
DATA = np.load("Experiment1/Words/" + articulationManner + subject + ".npy")
   
mean = np.mean(DATA, axis = -1)
std = np.std(DATA, axis = -1)
DATA = (DATA - mean[..., np.newaxis])/(std[..., np.newaxis] + 1e-5)

wordMatrices = DATA
labelsByWords = np.array([[i] * trialsPerWord for i in range(numberWords)]).reshape(numberTrials)

Indices =  {}
for i in range(numberWords):
    Indices[i] = []
for i in range(len(labelsByWords)):
    Indices[labelsByWords[i]].append(i)

covariancesLabels = np.zeros((numberWords, trialsPerWord, numberChannels, numberChannels))
for i in range(numberWords):
    for j in range(trialsPerWord):
        covariancesLabels[i, j] = 1/windowLength * (wordMatrices[Indices[i][j]] @ wordMatrices[Indices[i][j]].T)

trainFeatures = np.zeros((numberWords * 6, numberChannels, numberChannels))
trainLabels = np.zeros((numberWords * 6))
count = 0
for i in range(numberWords):
    trainFeatures[count:count + 3] = covariancesLabels[i, :3]
    trainFeatures[count + 3:count + 6] = covariancesLabels[i, 5:8]
    trainLabels[count:count + 6] = [i] * 6
    count += 6

testFeatures = np.zeros((numberWords * 4, numberChannels, numberChannels))
testLabels = np.zeros((numberWords * 4))
count = 0
for i in range(numberWords):
    testFeatures[count:count + 2] = covariancesLabels[i, 3:5]
    testFeatures[count + 2:count + 4] = covariancesLabels[i, 8:10]
    testLabels[count:count + 4] = [i] * 4
    count += 4

In [8]:
trainDataset = BaseDataset(trainFeatures, trainLabels)
testDataset = BaseDataset(testFeatures, testLabels)
trainDataloader = DataLoader(trainDataset, batch_size = 32, shuffle = True)
testDataloader = DataLoader(testDataset, batch_size = 32, shuffle = False)

In [9]:
numberEpochs = 1000

model = spdNet.learnSPDMatrices(numberWords).to(device)
numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(numParams)

lossFunction = nn.CrossEntropyLoss()
spdOptimizer = optimizers.MixOptimizer(model.parameters(), lr = 0.05)

10496


In [10]:
maxValue = 0
for epoch in range(numberEpochs):
    trainLoss, trainAccuracy = trainTest.trainOperation(model, device, trainDataloader, spdOptimizer, lossFunction)
    testLoss, testAccuracy = trainTest.testOperation(model, device, testDataloader, lossFunction)
    if maxValue < testAccuracy:
        maxValue = testAccuracy
    print(f'Epoch: {epoch + 1}/{numberEpochs}, Training loss: {trainLoss:.4f}, Training accuracy: {trainAccuracy:.2f}%, Test loss: {testLoss:.4f}, Test accuracy: {testAccuracy:.2f}%')
print(maxValue)

Epoch: 1/1000, Training loss: 0.1182, Training accuracy: 1.85%, Test loss: 0.1245, Test accuracy: 3.47%
Epoch: 2/1000, Training loss: 0.1173, Training accuracy: 1.85%, Test loss: 0.1238, Test accuracy: 4.17%
Epoch: 3/1000, Training loss: 0.1167, Training accuracy: 5.09%, Test loss: 0.1232, Test accuracy: 6.25%
Epoch: 4/1000, Training loss: 0.1160, Training accuracy: 5.09%, Test loss: 0.1228, Test accuracy: 6.25%
Epoch: 5/1000, Training loss: 0.1154, Training accuracy: 6.48%, Test loss: 0.1223, Test accuracy: 8.33%
Epoch: 6/1000, Training loss: 0.1148, Training accuracy: 7.41%, Test loss: 0.1219, Test accuracy: 8.33%
Epoch: 7/1000, Training loss: 0.1143, Training accuracy: 7.87%, Test loss: 0.1214, Test accuracy: 11.81%
Epoch: 8/1000, Training loss: 0.1140, Training accuracy: 6.48%, Test loss: 0.1210, Test accuracy: 9.72%
Epoch: 9/1000, Training loss: 0.1134, Training accuracy: 11.11%, Test loss: 0.1205, Test accuracy: 13.19%
Epoch: 10/1000, Training loss: 0.1127, Training accuracy: 10.

In [11]:
print(maxValue)

72.91666666666667


In [ ]:
%autoreload